# Aprendizagem por reforço: jogo da velha

 Número de posições possíveis com pelo menos duas casas livres = $3^7 \times 72 = 157.464$ (número de possibilidades para um tabuleiro com $7$ casas, e $3$ possibilidades por casa - O, X e vazio - vezes o número de maneiras possíveis de escolher as duas casas livres)
 
## Modelo

Para uma certa posição $x$, com $k(x)$ casas livres, tenho $n(x)$ ações possíveis. A partir de agora, vou ignorar o $x$, mantendo sempre em mente que teremos um modelo desses abaixo para cada uma das $157.464$ posições.

Para cada uma das $k$ ações possíveis, defino uma probabilidade $\pi(i)$: a probabilidade de vitória caso o jogador escolha a a ação $i$ ($i=1,...k$)

Observo $n_i$ jogos começando da mesma posição, e escolho sempre a ação $i$. Conto o número de vitórias $v_i$. Repito isso para todas as $k$ posições; ou seja, observo $(v_i, n_i)$ para $i=1,...,k$.  

Supondo que conheço as probabilidades $\pi(i)$, a probabilidade associada ao número de jogos e vitórias observadas (a verossimilhança) é uma binomial:

$P(v_i | n_i, \pi(i)) \propto \pi(i)^{v_i}(1-\pi(i))^{n_i-v_i}$

Considerando agora todas as posições, e supondo que os resultados dos jogos são independentes (condicionalmente a $\pi$), temos a verossimilhança conjunta de todas as posições:

$P(\mathbf{v} | \mathbf{n}, \mathbf{\pi}) \propto \prod_{i=1}^k \pi(i)^{v_i}(1-\pi(i))^{n_i-v_i}$

Como não conheço $\pi$, vou modelar minha ignorância usando uma distribuição de probabilidade para esses valores (uma priori). Por questões matemáticas, escolho uma distribuição Beta para cada possível ação:

$P(\pi(i)) \propto \pi(i)^{\alpha_i - 1}(1-\pi(i))^{\beta_i-1}$

Onde $\alpha_i$ e $\beta_i$ são hiperparâmetros que vamos precisar escolher (em lugar de aprender seus valores). 

## Aprendizagem 

Eu começo então dizendo que a probabilidade de vitória do movimento $i$ tem distribuição a priori Beta com hiperparâmetros $\alpha_i$ e $\beta_i$; jogo uma vez e escolho o movimento $i$. Como incorporo essa nova informação?

Pelo teorema de Bayes, e usando a propriedade de conjugação entre priori Beta e verossimilhança binomial, sei que a nova distribuição de $\pi(i)$ é de novo uma Beta, mas agora com parâmetros $(\alpha_i + 1, \beta_i)$ em caso de vitória, e $(\alpha_i, \beta_i+1)$ em caso de derrota. 

Portanto a equação de aprendizagem desse modelo, para cada posição $x$ e cada possível movimento $i_x$, é:

$\begin{align}
&\alpha_i(t+1) = \alpha_i(t) + r_i(t) \\
&\beta_i(t+1) = \beta_i(t) + (!-r_i(t))
\end{align}$

onde $r_i(t) = 1$ no caso de vitória, e $0$ caso contrário.

Observação: agora temos uma interpretação para os hiperparâmetros $\alpha$ e $\beta$; eles representam quantas vitórias e quantas derrotas o robô viu até o momento em que começa a nova rodada de aprendizagem.

## Decisão

O robô chegou numa posição $x$ com diversas ações possíveis. Ele tem uma distribuição de probabilidade para a probabilidade de vitória (a probabilidade de uma probabilidade...) de cada lance. O que fazer?

1. Olhar o valor esperado de probabilidade de vitória para cada movimento; escolher o movimento com maior valor esperado.
2. Sortear uma probabilidade de vitória para cada movimento, conforme a distribuição atual; escolher o movimento com maior valor sorteado

In [467]:
%matplotlib tk
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

class owg:
    def __init__(self):
        self.state = np.empty(shape = (3, 3))
        self.cur = None
        self.state[:] = np.nan
        self.__convert_state()
        self.starter = 0

    def reset(self):
        self.state = np.empty(shape = (3, 3))
        self.state[:] = np.nan
        self.__convert_state()
        self.starter = 1 - self.starter
        self.cur = None
        
    def __check_move(self, i, j):
        return np.isnan(self.state[i,j])
    
    def play(self, player, movimento):
        i = movimento[0]
        j = movimento[1]
        
        if not self.__check_move(i,j):
            #raise ValueError("Erro! movimento ({},{}) não permitido".format(i,j))
            print("Erro! movimento ({},{}) não permitido".format(i,j))
            return False
        
        if player is None:
            if self.cur is None:
                self.cur = 0
            player = self.cur
            
        if player not in [0,1]:
            #raise ValueError("Erro! jogador deve ser 0 ou 1")
            print("Erro! jogador deve ser 0 ou 1")
            return False
        
        if self.cur is None:
            self.cur = player
        else:
            if self.cur != player:
                #raise ValueError("Erro! Não é a vez do jogador {}".format(player))
                print("Erro! Não é a vez do jogador {}".format(player))
                return False
        
        self.state[i,j] = player
        self.__convert_state()
        self.cur = 1 - self.cur
        return True
        
    def check_result(self):
        # Checa se o jogo acabou
        # Soma as colunas
        colsum = self.state.sum(axis = 1)
        if np.any(colsum == 3):
            # 1 ganhou
            i = int(np.where(colsum == 3)[0])
            return 1, (3,i)
        elif np.any(colsum == 0):
            # 0 ganhou
            i = int(np.where(colsum == 0)[0])
            return -1, (3,i)
        rowsum = self.state.sum(axis= 0)
        if np.any(rowsum == 3):
            # 1 ganhou
            i = int(np.where(rowsum == 3)[0])
            return 1, (i,3)
        elif np.any(rowsum == 0):
            # 0 ganhou
            i = int(np.where(rowsum == 0)[0])
            return -1, (i,3)
        # Diagonal
        diag = self.state[0,0] + self.state[1,1] + self.state[2,2]
        if diag == 0:
            return -1, (-3,-3)
        elif diag == 3:
            return 1, (-3,-3)
        
        # Segunda diagonal
        diag = self.state[0,2] + self.state[1,1] + self.state[2,0]
        if diag == 0:
            return -1, (3,3)
        elif diag == 3:
            return 1, (3, 3)
        
        
        if np.any(np.isnan(self.state)):
            # ainda não acabou
            return None, None
        else:
            # empatou
            return 0, None
        
    def __convert_state(self):
        # Convert state to integer on base 3
        # 0 = O, 1 = X, None = empty position
        istate = 0
        tmpstate = self.state.copy()
        tmpstate[np.isnan(tmpstate)] = 2
        tmpstate = tmpstate.flatten().astype(int)

        sstate = np.array2string(tmpstate, separator = "", prefix= "")[1:-1]
            
        self.istate = int(sstate, 3)
        self.sstate = sstate
        
    def start(self):
        # Tabuleiro para human x human
        N = 2000
        Nbola = 10000
        # Gera conjunto de pontos para as figuras
        # X
        xx = np.random.normal(loc = 0, scale = 0.05, size = N)
        yx1 = 2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 
        yx2 = -2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 

        # Bola
        xb = np.random.normal(loc = 0, scale = 0.05, size = Nbola)
        yb = np.random.normal(scale = 0.1, size = len(xb)) 

        self.reset()
        
        def onclick(self, event):
            ix, iy = event.xdata, event.ydata
            cur = tab.cur

            r, tipo = self.check_result()

            if r is not None:
                plt.cla()
                plt.vlines(x = 1/2, ymin = 0, ymax = 3)
                plt.vlines(x = 1, ymin = 0, ymax = 3)
                plt.hlines(y = 1, xmin = 0, xmax = 3/2)
                plt.hlines(y = 2, xmin = 0, xmax = 3/2)
                plt.axis('off')
                self.reset()
            else:

                if ix > 0.0 and ix < 1.5:
                    if ix <= 0.5:
                        i = 0
                    elif ix <= 1.:
                        i = 1
                    else:
                        i = 2
                else:
                    i = None

                if iy > 0.0 and iy <= 3:
                    if iy <= 1.:
                        j = 0
                    elif iy <= 2:
                        j = 1
                    else:
                        j = 2
                else:
                    j = None

                if i is not None and j is not None:
                    res = self.play(None, (i,j))
                    if res:
                        # Centro do quadrado
                        xc = i/2 + .235
                        yc = 0.5 + j 
                        if tab.cur:
                            x = xx + xc
                            y = yx1 + yc
                            y2 = yx2 + yc                    
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                            plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                        else:
                            x = xb + xc
                            y = yb + yc
                            z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                            x = [zz[0] for zz in z]
                            y = [zz[1] for zz in z]
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')


                    r, tipo = self.check_result()

                    if r is not None:
                        if r == 0:
                            print("Empate")
                        elif r == 1:
                            print("X ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        
                        elif r == -1:
                            print("O ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        

            plt.show()


        fig = plt.figure(figsize = (10, 10))

        plt.vlines(x = 1/2, ymin = 0, ymax = 3)
        plt.vlines(x = 1, ymin = 0, ymax = 3)
        plt.hlines(y = 1, xmin = 0, xmax = 3/2)
        plt.hlines(y = 2, xmin = 0, xmax = 3/2)
        plt.axis('off')

        cid = fig.canvas.mpl_connect('button_press_event', lambda l:onclick(self, l))


        plt.show()        
        
        
    def start(self, p1):
        # Tabuleiro para human x computer
        p1.reset()
        N = 2000
        Nbola = 10000
        # Gera conjunto de pontos para as figuras
        # X
        xx = np.random.normal(loc = 0, scale = 0.05, size = N)
        yx1 = 2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 
        yx2 = -2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 

        # Bola
        xb = np.random.normal(loc = 0, scale = 0.05, size = Nbola)
        yb = np.random.normal(scale = 0.1, size = len(xb)) 

        self.reset()
        
        def onclick(self, event):
            ix, iy = event.xdata, event.ydata

            r, tipo = self.check_result()

            if r is not None:
                plt.cla()
                plt.vlines(x = 1/2, ymin = 0, ymax = 3)
                plt.vlines(x = 1, ymin = 0, ymax = 3)
                plt.hlines(y = 1, xmin = 0, xmax = 3/2)
                plt.hlines(y = 2, xmin = 0, xmax = 3/2)
                plt.axis('off')
                self.reset()
                self.cur = self.starter
                p1.reset()
                
                if self.starter == 1:
                    movimento = p1.joga()

                    if movimento is not None:
                        res = self.play(None, movimento)
                        i = movimento[0]
                        j = movimento[1]
                        if res:
                            # Centro do quadrado
                            xc = i/2 + .235
                            yc = 0.5 + j 
                            if self.cur:
                                x = xx + xc
                                y = yx1 + yc
                                y2 = yx2 + yc                    
                                plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                                plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                            else:
                                x = xb + xc
                                y = yb + yc
                                z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                                x = [zz[0] for zz in z]
                                y = [zz[1] for zz in z]
                                plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')                
            else:

                if ix > 0.0 and ix < 1.5:
                    if ix <= 0.5:
                        i = 0
                    elif ix <= 1.:
                        i = 1
                    else:
                        i = 2
                else:
                    i = None

                if iy > 0.0 and iy <= 3:
                    if iy <= 1.:
                        j = 0
                    elif iy <= 2:
                        j = 1
                    else:
                        j = 2
                else:
                    j = None

                if i is not None and j is not None:
                    res = self.play(None, (i,j))
                    if res:
                        # Centro do quadrado
                        xc = i/2 + .235
                        yc = 0.5 + j 
                        if self.cur:
                            x = xx + xc
                            y = yx1 + yc
                            y2 = yx2 + yc                    
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                            plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                        else:
                            x = xb + xc
                            y = yb + yc
                            z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                            x = [zz[0] for zz in z]
                            y = [zz[1] for zz in z]
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')
                        

                    r, tipo = self.check_result()

                    if r is not None:
                        if r == 0:
                            print("Empate")
                        elif r == 1:
                            print("X ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        
                        elif r == -1:
                            print("O ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        

                p1.comunica((i,j))     
                movimento = p1.joga()

                if movimento is not None:
                    res = self.play(None, movimento)
                    i = movimento[0]
                    j = movimento[1]
                    if res:
                        # Centro do quadrado
                        xc = i/2 + .235
                        yc = 0.5 + j 
                        if self.cur:
                            x = xx + xc
                            y = yx1 + yc
                            y2 = yx2 + yc                    
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                            plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                        else:
                            x = xb + xc
                            y = yb + yc
                            z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                            x = [zz[0] for zz in z]
                            y = [zz[1] for zz in z]
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')


                    r, tipo = self.check_result()

                    if r is not None:
                        if r == 0:
                            print("Empate")
                        elif r == 1:
                            print("X ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        
                        elif r == -1:
                            print("O ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                                                        


            plt.show()


        fig = plt.figure(figsize = (10, 10))

        plt.vlines(x = 1/2, ymin = 0, ymax = 3)
        plt.vlines(x = 1, ymin = 0, ymax = 3)
        plt.hlines(y = 1, xmin = 0, xmax = 3/2)
        plt.hlines(y = 2, xmin = 0, xmax = 3/2)
        plt.axis('off')
        
        
        if self.starter == 0:
            movimento = p1.joga()

            if movimento is not None:
                res = self.play(None, movimento)
                i = movimento[0]
                j = movimento[1]
                if res:
                    # Centro do quadrado
                    xc = i/2 + .235
                    yc = 0.5 + j 
                    if self.cur:
                        x = xx + xc
                        y = yx1 + yc
                        y2 = yx2 + yc                    
                        plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                        plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                    else:
                        x = xb + xc
                        y = yb + yc
                        z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                        x = [zz[0] for zz in z]
                        y = [zz[1] for zz in z]
                        plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')

        cid = fig.canvas.mpl_connect('button_press_event', lambda l:onclick(self, l))


        plt.show()                

In [450]:
class owg_player:
    def __init__(self):
        # board é um objeto owg
        # knowledge é um dicionário 'posicao' : [probabilidades], que atribui probabilidades a cada movimento na posição posicao
        # O jogador sempre se considera o jogador 1
        self.knowledge = dict()
        self.board = owg()
        self.jogo = []
        
    def comunica(self, movimento, verbose = False):
        # Comunica o movimento do oponente
        self.board.play(0, movimento)
        if verbose:
            print("Comunica", movimento, self.board.state)


    def __inicializa(self, strpos):
        pass
    
    def joga(self, verbose = False):
        pass
    
    def reset(self):
        self.board.reset()
        self.jogo = []
    # Para cada possível posição, uma dist de probabilidade sobre as posições restantes
    # Represento a matriz como um inteiro entre 0 e 2^8, para facilitar a busca 

In [451]:
class jb (owg_player) : 
    ''' Classe jb: não aprende; decide cada movimento aleatoriamente para sempre
    '''
    def __init__(self):
        self.nome = 'JB'
        owg_player.__init__(self)
        
    def __inicializa(self, strpos):
        # Cria o vetor de probabilidade uniforme para a dada posição
        acoes = [i for i in range(len(strpos)) if strpos[i] == '2']
        probs = [1/len(acoes)] * len(acoes)
        self.knowledge[strpos] = (acoes, probs)
    
    
    def joga(self, verbose = False):
        # Dada a posição atual do tabuleiro, decide a ação
        # PRimeiro verifica se jogo está rolando
        r, _ = self.board.check_result()
        if r is None:

            strpos = self.board.sstate
            if verbose:
                print(strpos)
            if strpos not in self.knowledge.keys():
                # Nunca viu essa posição
                self.__inicializa(strpos)

            acoes, probs = self.knowledge[strpos]
            acao = np.random.choice(a = acoes, p = probs)

            movimento = (int(acao / 3), acao % 3)
            
            #if verbose:
                #print(self.board.state, acao)
                #print(self.board.sstate, acoes)

            self.board.play(1, movimento)
            self.jogo.append((strpos, acao))
            
            return movimento
        else:
            return None

In [452]:
class cientista(owg_player):
    ''' Class cientista: usa o modelo probabilístico para aprender e atualizar as probabilidades
    '''
    def __init__(self, a = 1, b = 1):
        self.a = a
        self.b = b
        self.nome = 'Cientista'
        owg_player.__init__(self)
        
    def __inicializa(self, strpos):
        # Cria o vetor de probabilidade uniforme para a dada posição
        acoes = [i for i in range(len(strpos)) if strpos[i] == '2']
        alfa = [self.a] * len(acoes)
        beta = [self.b] * len(acoes)
        self.knowledge[strpos] = (acoes, alfa, beta)
        
    def joga(self, verbose = False):

        # Dada a posição atual do tabuleiro, decide a ação
        # PRimeiro verifica se jogo está rolando
        r, _ = self.board.check_result()
        if r is None:

            strpos = self.board.sstate
            if verbose:
                print(strpos)
            if strpos not in self.knowledge.keys():
                # Nunca viu essa posição
                self.__inicializa(strpos)

            acoes, alfa, beta = self.knowledge[strpos]
            probs = [np.random.beta(a = param[0], b = param[1]) for param in zip(alfa, beta)]
            acao = acoes[np.argmax(probs)]

            movimento = (int(acao / 3), acao % 3)
            
            #if verbose:
                #print(self.board.state, acao)
                #print(self.board.sstate, acoes)

            self.board.play(1, movimento)
            self.jogo.append((strpos, acao))
            
            # Verifica se ganhou o jogo
            r, _ = self.board.check_result()
            if r is not None:
                if r == 1:
                    # Distribui as recompensas
                    for i in range(len(self.jogo)):
                        pos = self.jogo[i][0]
                        acao = self.jogo[i][1]
                        a, al, be = self.knowledge[pos]
                        al[a.index(acao)] += 1
                        self.knowledge[pos] = (a, al, be)

            return movimento
        else:
            if r == -1:
                # Perdeu o jogo
                # Distribui as recompensas
                for i in range(len(self.jogo)):
                    pos = self.jogo[i][0]
                    acao = self.jogo[i][1]
                    a, al, be = self.knowledge[pos]
                    be[a.index(acao)] += 1
                    self.knowledge[pos] = (a, al, be)
                    
            return None        
    

In [453]:
class cientista_sovina(owg_player):
    ''' Class cientista: usa o modelo probabilístico para aprender e atualizar as probabilidades
    '''
    def __init__(self, a = 1, b = 1):
        self.a = a
        self.b = b
        self.nome = 'Cientista sovina'
        owg_player.__init__(self)
        
    def __inicializa(self, strpos):
        # Cria o vetor de probabilidade uniforme para a dada posição
        acoes = [i for i in range(len(strpos)) if strpos[i] == '2']
        alfa = [self.a] * len(acoes)
        beta = [self.b] * len(acoes)
        self.knowledge[strpos] = (acoes, alfa, beta)
        
    def joga(self, verbose = False):

        # Dada a posição atual do tabuleiro, decide a ação
        # PRimeiro verifica se jogo está rolando
        r, _ = self.board.check_result()
        if r is None:

            strpos = self.board.sstate
            if verbose:
                print(strpos)
            if strpos not in self.knowledge.keys():
                # Nunca viu essa posição
                self.__inicializa(strpos)

            acoes, alfa, beta = self.knowledge[strpos]
            probs = [param[0] / (param[0] + param[1]) for param in zip(alfa, beta)]
            acao = acoes[np.argmax(probs)]

            movimento = (int(acao / 3), acao % 3)
            
            #if verbose:
                #print(self.board.state, acao)
                #print(self.board.sstate, acoes)

            self.board.play(1, movimento)
            self.jogo.append((strpos, acao))
            
            # Verifica se ganhou o jogo
            r, _ = self.board.check_result()
            if r is not None:
                if r == 1:
                    # Distribui as recompensas
                    for i in range(len(self.jogo)):
                        pos = self.jogo[i][0]
                        acao = self.jogo[i][1]
                        a, al, be = self.knowledge[pos]
                        al[a.index(acao)] += 1
                        self.knowledge[pos] = (a, al, be)

            return movimento
        else:
            if r == -1:
                # Perdeu o jogo
                # Distribui as recompensas
                for i in range(len(self.jogo)):
                    pos = self.jogo[i][0]
                    acao = self.jogo[i][1]
                    a, al, be = self.knowledge[pos]
                    be[a.index(acao)] += 1
                    self.knowledge[pos] = (a, al, be)
                    
            return None      

In [454]:
p1 = cientista()
p2 = cientista()

vitorias = 0
derrotas = 0
empates = 0
pempate = []
pjog1 = []
pjog2 = []
n = 10000

for i in range(n):
    
    if i % 2 == 0:
        movimento = p1.joga()
        p2.comunica(movimento)
        res, _ = p2.board.check_result()
        while res is None:
            movimento = p2.joga()
            if movimento is not None:
                p1.comunica(movimento)
                movimento = p1.joga()           
                if movimento is not None:
                    p2.comunica(movimento)
            res, _ = p2.board.check_result()
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
    else:
        movimento = p2.joga()
        p1.comunica(movimento)
        res, _ = p1.board.check_result()
        while res is None:
            movimento = p1.joga()
            if movimento is not None:
                p2.comunica(movimento)
                movimento = p2.joga()          
                if movimento is not None:
                    p1.comunica(movimento)
            res, _ = p1.board.check_result()
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
        
    print("Jogo {}: P1 {:.2%}, P2 {:.2%}, Empate {:.2%}".format(i+1, vitorias / (i+1), derrotas / (i+1), empates / (i+1)))
    pempate.append(empates / (i+1))
    pjog1.append(vitorias / (i+1))
    pjog2.append(derrotas / (i+1))

Jogo 1: P1 100.00%, P2 0.00%, Empate 0.00%
Erro! Não é a vez do jogador 0
Erro! movimento (2,1) não permitido
Erro! Não é a vez do jogador 1
Erro! movimento (2,0) não permitido
Jogo 2: P1 50.00%, P2 0.00%, Empate 50.00%
Erro! Não é a vez do jogador 1
Jogo 3: P1 33.33%, P2 0.00%, Empate 66.67%
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro

Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro

Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jog

Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jog

Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez d

...

Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) nã

Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Err

Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez d

Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) não permitido
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! Não é a vez do jogador 0
Erro! Não é a vez do jogador 1
Erro! movimento (1,1) nã

KeyboardInterrupt: 

In [430]:
from matplotlib import pyplot as plt
plt.figure(figsize = (20, 10))
plt.plot(pjog1, 'o-', label = p1.nome)
plt.plot(pempate, 'o-', label = 'Empate')
plt.plot(pjog2, 'o-', label = p2.nome)
plt.ylim([0,1])
plt.hlines(y = 0.5, xmin = 0, xmax = len(pjog1) )
plt.grid(True)
plt.legend()
plt.show()

In [469]:
# Jogar contra quem?
p = p1
tab = owg()
tab.start(p)

O ganhou
Erro! Não é a vez do jogador 0
